# Notebook to select host galaxies and submit candidates to TNS

In [ ]:
import sys, os
import pandas as pd
import numpy as np
import ipyaladin as ipyal
from ipywidgets import Layout, Box, widgets
import json
import time
import pickle

In [ ]:
from astroquery.ned import Ned
from astroquery.simbad import Simbad
from astroquery.vizier import Vizier
import astropy.units as u
from astropy.table import Table, Column

In [ ]:
sys.path.append("./lib")
from alerce_tns import *
#from alerce_tns_Ale230426 import *

In [ ]:
#!export PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python

In [ ]:
from delight.delight import Delight

In [ ]:
# continuously load modules (for easier debugging)
%load_ext autoreload
%autoreload 2

# Start API

In [ ]:
alerce = alerce_tns()

# Load recent candidates

**WARNING: CHECK THE DATE**

In [ ]:
refstring = "20240213" #"20220815"
candidates_dir = "candidates"
candidates = open("%s/%s.csv" % (candidates_dir, refstring), encoding='utf-8-sig').read().splitlines()
if "" in candidates:
    candidates.remove("") 
candidates = np.sort(np.unique(np.array([cand.strip() for cand in candidates])))
print(candidates, len(candidates))


In [ ]:
stats = alerce.query_objects(oid=candidates, format='pandas', page_size=len(candidates))
delight_datadir = "delight_data"
dclient = Delight(delight_datadir, candidates, stats.meanra.to_numpy(), stats.meandec.to_numpy())
dclient.download()
dclient.get_pix_coords()
dclient.compute_multiresolution(nlevels=5, domask=False, doobject=True, doplot=False)
dclient.load_model()
print("Preprocessing")
dclient.preprocess()
print("Predicting")
dclient.predict()

In [ ]:
for oid in candidates:
    dclient.plot_host(oid)

# Select host galaxies using aladin widget

## Do automatic selection

In [ ]:
bestguess = dclient.df[["ra_delight", "dec_delight"]].rename({"ra_delight": "ra", "dec_delight": "dec"}, axis=1)
alerce.bestguess = bestguess

In [ ]:
# if ned, simbad or sdssdr16 apis fail disable their option
alerce.select_hosts(candidates, refstring, ned=True, simbad=True, SDSSDR16=True, catsHTM=False, vizier=False)

## Do manual selections for errors

Errors could happen because of an error in DELIGHT or because the association favoured a more distant host with redshift.

In [ ]:
# interesting example that works well ZTF23aaqocso
#["ZTF23aavmgln"]#'ZTF23aatvump', "ZTF23aatzjyz"]#ZTF23aatpmpk']#"ZTF23aaqyrha", "ZTF23aarvadj"]#"ZTF23aaqpzom", "ZTF23aaqqfac"] #"ZTF23aaqctty"]#"ZTF23aapssmi", "ZTF23aapthmh"]#"ZTF23aanlgap"]#"ZTF23aampyvc"]#"ZTF23aamqklj", "ZTF23aamscqd"]#'ZTF23aajgivv'] #'ZTF23aaiyxbx', "ZTF23aaiztxa"]#"ZTF23aailczv"]#"ZTF23aagaimn"] #"ZTF22abvvpcm", "ZTF23aafwzot"]#"ZTF23aaelbet", "ZTF23aafqlnn"] #"ZTF23aaelqpq", "ZTF23aaenqok"]#"ZTF23aadvjqb", "ZTF23aaeedgu"]#"ZTF23aaaoiap", "ZTF23aaapqrh"] #"ZTF23aaalucb"]#"ZTF22abyzowo"]#"ZTF22abyodaq"]#"ZTF22abtoqfj", "ZTF22abuqqxy"]#"ZTF22aboawxv"]#"ZTF22abpmlkx", "ZTF22abqtdyh", "ZTF22abrahwy", "ZTF22abrezqt", "ZTF22abrfsba", "ZTF22abrftlp", "ZTF22abrhrbn"]#"ZTF22abnvljq", "ZTF22abnyjut", "ZTF22abpqcoj"]#"ZTF22abivrnq", "ZTF22abjrcte", "ZTF22abjuhua"]#"ZTF22abhklzv", "ZTF22abhxqgv", "ZTF22abickzh"]#"ZTF22abhclyh", "ZTF22abhudnw", "ZTF22abhvfmz"]#"ZTF22abgkeqz"] #ZTF22abfzvai", "ZTF22abgdolw", "ZTF22abghfyi"]#ZTF22abfdiwm"]#"ZTF22aazuomx"]#ZTF22aazkhnu"]#"ZTF22aayylvj", "ZTF22aazaxcu"]#"ZTF22aaxafin"]#"ZTF22aasxnjm", "ZTF22aatrnbl", "ZTF22aattadv"] #"ZTF22aaujkdb", "ZTF22aaveghe"]
errors = ["ZTF24aafbfgj"] #"ZTF24aaekaaj", "ZTF24aaekogp"]#["ZTF24aaaajjt"] #["ZTF23abvotgv"]#"ZTF23abteyml"]#"ZTF23abooocr"]#"ZTF23abpbdfm"]#"ZTF23aajlkxc"]#"ZTF23abjtxza", "ZTF23abjwfjb"]#"ZTF23abjgjnm"] #"ZTF23abjfiiw"]#"ZTF23abivqyc"]#"ZTF23abdgsbw", "ZTF23abdyqcv"]#"ZTF23aazuzds"]#, "ZTF23aazzgxk"]
# confirm if this was a problem with DELIGHT
#for oid in errors:
#    dclient.plot_host(oid)
# do manual selection
if errors != []:
    alerce.select_hosts(errors, refstring, ned=True, simbad=True, catsHTM=False, vizier=False, dobestguess=False)

## Manual modifications

In [ ]:
#oid = "ZTF18acbycde"#oid = "ZTF21ablxqtj"
#oid = "ZTF21acbcfmh" #ZTF21abidhcz"
##oid = 'ZTF20aclzygy'#ZTF20acgigfo'
##oid = "ZTF20aaqrmxg" #"ZTF20aakdppm" #ZTF19acxrihd"
#oid = 'ZTF22aaajgss' #ZTF21aaryakx' #ZTF20acvbneg'
#alerce.candidate_hosts.host_name[oid] = "M31"
#alerce.candidate_hosts.host_ra[oid] = ""
#alerce.candidate_hosts.host_dec[oid] = ""
#alerce.candidate_hosts.host_offset[oid] = ""
#alerce.candidate_hosts.host_source[oid] = ""
#alerce.candidate_hosts.host_offset_bestguess[oid] = ""
#alerce.candidate_hosts.bestguess_offset[oid] = ""
#alerce.candidate_hosts.host_redshift_spec[oid] = True
#alerce.candidate_hosts.host_redshift[oid] = = "0.001341"
#alerce.candidate_hosts.host_redshift_error[oid] = "0.000040"
#alerce.candidate_hosts.host_redshift_type[oid] = "specz"
# #"nan" #"0.041" #nan"
##for i in list(alerce.candidate_hosts):
##    alerce.candidate_hosts.loc['ZTF21aaryakx'][i] = "NULL"

## Save hosts

In [ ]:
alerce.candidate_hosts.to_csv("hosts/%s_hosts.csv" % refstring)
alerce.candidate_hosts.head(100)

## Show nearest sources (z < 0.01)

In [ ]:
alerce.candidate_hosts.host_redshift

In [ ]:
alerce.candidate_hosts.host_redshift.loc[alerce.candidate_hosts.host_redshift == '--'] = "NULL"

In [ ]:
zth = 0.1
nearest = pd.to_numeric(alerce.candidate_hosts.loc[alerce.candidate_hosts.host_redshift != "NULL"].host_redshift).sort_values()
nearest = nearest.loc[nearest < zth]
for i in ["https://alerce.online/object/%s %s" % (i, alerce.candidate_hosts.host_redshift.loc[i]) for i in nearest.index]:
    print(i)

# Prepare TNS report

## Sandbox or real

In [ ]:
sandbox = False #False # True: test in sandbox, False: send to real website

## Get API key and url

In [ ]:
if sandbox:
    print("Getting API sandbox key...")
    api_key = open("API_sandbox.key", "r").read()
    api_key = open("API.key", "r").read() # sandbox key appears not to be working
else:
    print("Getting API key...")
    api_key = open("API.key", "r").read()
    
if sandbox:
    url_tns_api = "https://sandbox.wis-tns.org/api" #"https://www.wis-tns.org/" #"https://sandbox-tns.weizmann.ac.il/api"
else:
    url_tns_api = "https://www.wis-tns.org/api" #"https://wis-tns.weizmann.ac.il/api" #"https://www.wis-tns.org/" #"https://wis-tns.weizmann.ac.il/api"
    
if not sandbox:
    print("WARNING: submitting to real (not sandbox) TNS website")

## Confirm reporter names

This is also used for Skyportal

In [ ]:
reporter = "F. Forster, F.E. Bauer, G. Pignata, A. Munoz-Arancibia, A. Mourao, L. Hernandez-Garcia, J.P. Brandt, L. Galbany, J. Silva-Farfan, R. Dastidar, E. Camacho, A. Alvarez, G. Cabrera-Vives, R. Carrasco-Davis, P.A. Estevez, P. Gallardo, P. Huijse, H. Larranaga, E. Pizarro, I. Reyes, P. Sanchez-Saez, A. Sepulveda, D. Rodriguez-Mancini, M. Catelan, S. Eyheramendy, M.J. Graham on behalf of the ALeRCE broker"

## Build the report

In [ ]:
# first author: the person submitting, 2nd author: those that vetoed, then everyone else
tns_report = {"at_report": {}}
counter = 0
for oid in candidates[::-1]: # invert to report old candidates first
    #if oid in ["ZTF20aauqmwf"]:#["ZTF19acgjpfd", "ZTF18acszatf", "ZTF18acrkmnm"]:
    #    continue
    #if oid in ["ZTF20acqnfkh ", "ZTF20acneabh"]:#ZTF20acplctd":#aamtgd": #ZTF19acymtgr": #ZTF19acnqqhl":#in ["ZTF19acmdwjy", "ZTF19acmdwjz"]:
    #    continue
    #if oid == "ZTF20ablnjwh": #ZTF20abkmxrm":#ZTF20abgfgne":
    #    continue
    if oid in ["ZTF23aafxbvj", "ZTF22aabrskg"]: #ZTF21abgvidd"]: #ZTF21aawgsfp"]:#ZTF21aalcrjh"]: #ZTF21aaanwxo"]:#, ZTF20adadixi"]:# ZTF20acsggey"]:#, "ZTF20acmevcc", "ZTF20acmctng", "ZTF20acmdvzk"]: #== "ZTF20acgknep":#"ZTF20acdcdqq": #ZTF20acaehey": #ZTF20abyjoxb":#"ZTF20abqjddh":#ZTF20abkxqzv":#ZTF20abgbfyk":
        continue

    print("\n", counter, oid)
    sn_report = alerce.do_TNS_report(api_key, oid, reporter, verbose=False, test=False)
    if sn_report:# or oid == "ZTF20abyptpc":
        tns_report["at_report"]["%i" % counter] = sn_report
        counter += 1


## Manual modifications

In [ ]:
#report['at_report']['0']['remarks'] = "Fast rising transient (g-rise > 1.48 mag/day)"#report['at_report']['3']['remarks'] = 'Nova candidate in M31 (g-rise > 0.16+-0.01 mag/day, r-rise > 0.24+-0.01 mag/day) classified by ALeRCE using the public ZTF stream. Discovery image and light curve in http://alerce.online/object/ZTF19acxrihd '

In [ ]:
#oid = "ZTF21ablxqtj"
#for i in tns_report["at_report"].keys():
#    if tns_report["at_report"][i]["internal_name"] == oid:
#        tns_report["at_report"][i]["remarks"] = tns_report["at_report"][i]["remarks"].replace("SN candidate", "Nova candidate in M31")
#        print(tns_report["at_report"][i]["remarks"])

## Display report

In [ ]:
print(json.dumps(tns_report, indent=4, sort_keys=True))

## Look for fast risers

In [ ]:
for i in tns_report["at_report"]:
    if tns_report["at_report"][i]["remarks"][:5] == "Early":
        try:
            for rise in re.findall("rise\s>\s(.*?)\+-.*?\smag", tns_report["at_report"][i]["remarks"]):
                if float(rise) >= 0.25:
                    print(tns_report["at_report"][i]["remarks"])
                    continue
        except:
            rise = float(re.findall("rise\s>\s(.*?)\s", tns_report["at_report"][i]["remarks"])[0])
            if rise >= 0.25:
                print(tns_report["at_report"][i]["remarks"])

## Save report

In [ ]:
json_report = '%s/%s.json' % (candidates_dir, refstring)
with open(json_report, 'w') as outfile:
    json.dump(tns_report, outfile)

## Send to TNS

**WARNING: key is used to send candidates!**

In [ ]:
json_report

In [ ]:
r = alerce.send_json_report(api_key, url_tns_api, json_report)

## Show output

In [ ]:
r.content

## Show feedback

In [ ]:
# function for getting reply from report                             
def reply(url, report_id):                                           
    try:                                                               
        # url for getting report reply                                   
        reply_url=url+'/bulk-report-reply'                               
        # construct list of (key,value) pairs                                
        reply_data=[('api_key',(None, api_key)),                         
                    ('report_id',(None,report_id))]                      
        # send report ID using request module                            
        response=requests.post(reply_url, files=reply_data, headers=alerce.tns_headers)              
        # return response                                                
        return response                                                  
    except Exception as e:                                             
        return [None,'Error message : \n'+str(e)]  

In [ ]:
try:
    feedback = reply(url_tns_api, r.json()["data"]["report_id"]).json()["data"]["feedback"]
    print(feedback)
except:
    print("Report not available, probably being processed. Please try again...")

## Check the following link and confirm that the latest SNe are from the reported day

https://www.wis-tns.org/search?&discovered_period_value=1&discovered_period_units=days&reporting_groupid[]=74&discovery_data_source_id=48&num_page=50&format=html&edit[type]=&edit[objname]=&edit[id]=&display[redshift]=1&display[hostname]=1&display[host_redshift]=1&display[source_group_name]=1&display[classifying_source_group_name]=1&display[discovering_instrument_name]=0&display[classifing_instrument_name]=0&display[programs_name]=0&display[internal_name]=1&display[isTNS_AT]=0&display[public]=1&display[end_pop_period]=0&display[spectra_count]=1&display[discoverymag]=1&display[discmagfilter]=1&display[discoverydate]=1&display[discoverer]=1&display[remarks]=0&display[sources]=0&display[bibcode]=0&display[ext_catalogs]=0&sort=desc&order=discoverydate

Ignore everything from here

# Prepare SkyPortal report

In [ ]:
alerce.candidate_hosts = pd.read_csv("hosts/%s_hosts.csv" % refstring)
alerce.candidate_hosts.set_index("oid", inplace=True)
alerce.candidate_hosts.fillna("NULL", inplace=True)
alerce.candidate_hosts

## Load credentials

In [ ]:
skyportal_url = "http://desi2.lbl.gov:5000/api"
credentials_file = "credentials_skyportal.json"
with open(credentials_file) as jsonfile:
    params = json.load(jsonfile)
token = params["token"]

## Build the report

In [ ]:
skyportal_report = []
counter = 0
for oid in candidates[::-1]: # invert to report old candidates first
    if oid in ["ZTF21abkvttz"]: #ZTF21abhpdgd", "ZTF21abgtlir"]:
        continue
    print("\n", counter, oid)
    sn_report = alerce.do_skyportal_report(skyportal_url, token, oid, reporter, verbose=False, test=False)
    if sn_report:# or oid == "ZTF20abyptpc":
        skyportal_report.append(sn_report)
        counter += 1

## Save report

In [ ]:
with open('%s_skyportal.pickle' % refstring, 'wb') as handle:
    pickle.dump(skyportal_report, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('%s_skyportal.pickle' % refstring, 'rb') as handle:
    skyportal_report = pickle.load(handle)
    
display(skyportal_report)

## Submit to SkyPortal

In [ ]:
for report in skyportal_report:
    print(report["candidates"]["id"])
    alerce.send_skyportal_report(skyportal_url, token, report)

## Check SkyPortal

http://desi2.lbl.gov:5000/candidates